In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys

In [3]:
from micrograd.mnist import load_mnist
from micrograd.engine import Value
from micrograd.nn import MLP

In [2]:
sys.setrecursionlimit(10000)
BATCH_SIZE = 32

In [ ]:
(x_train, t_train), (x_test, t_test) = load_mnist()

print("Dataset dimension:")

print("Training set = ", len(x_train), len(t_train))
print("Test set = ", len(x_test), len(t_test))

print("Dimension of input = ", len(x_train[0]))

In [ ]:
num_row = 2
num_col = 5
num = num_row * num_col

fig, axes = plt.subplots(num_row, num_col, figsize = (1.5 * num_col, 2 * num_row))
for i in range(num):
    image = np.array(x_train[i], dtype='float').reshape((28, 28))
    ax = axes[i//num_col, i%num_col]
    ax.imshow(image, cmap='gray')
    ax.set_title('Label: {}'.format(t_train[i]))
plt.tight_layout()
plt.show()
plt.savefig("C:\\Users\\hosca\\Documents\\MNIST\\images\\image1.pdf", )


In [ ]:
model = MLP(784, [64, 10])
print('number of parameters', len(model.parameters()))
print(model)

In [ ]:
# loss function
def loss(batch_size=None):
    
    # inline DataLoader :)
    if batch_size is None:
        Xb, yb = x_train, t_train
    else:
        ri = np.random.permutation(x_train.shape[0])[:batch_size]
        Xb, yb = x_train[ri], t_train[ri]
    inputs = [list(map(Value, xrow)) for xrow in Xb]
    
    # forward the model to get scores
    scores = list(map(model, inputs)) 
    
    # Cross-entropy Loss
    losses = [(- score[yi].log()) for yi, score in zip(yb, scores)]
    total_loss = sum(losses) / len(losses)

    # Accuracy
    accuracy = []
    for yi, score in zip(yb, scores):
        argmax = -1
        valmax = 0
        for idx, scorei in enumerate(score):
            if scorei.data > valmax:
                valmax = scorei.data
                argmax = idx
        accuracy.append((argmax == yi))

    acc = sum(accuracy) / len(accuracy)
        
    return total_loss, acc

total_loss, acc = loss(batch_size = BATCH_SIZE)
print(f"step {0} loss {total_loss.data}, accuracy {acc*100}%")

In [ ]:
# backward
for k in range(100):
    model.zero_grad()
    total_loss.backward()
        
    # update (sgd)
    learning_rate = 1.0 - 0.9*k/100
    for p in model.parameters():
        p.data -= learning_rate * p.grad

    total_loss, acc = loss(batch_size = BATCH_SIZE)
        
    print(f"step {k + 1} loss {total_loss.data}, accuracy {acc*100}%")

In [10]:
def test(batch_size=None):
    
    ri = np.random.permutation(x_test.shape[0])[:batch_size]
    Xb, yb = x_test[ri], t_test[ri]
    inputs = [list(map(Value, xrow)) for xrow in Xb]
    
    # forward the model to get scores
    scores = list(map(model, inputs))
    
    # Cross-entropy Loss
    losses = [(- score[yi].log()) for yi, score in zip(yb, scores)]
    total_loss = sum(losses) / len(losses)

    # Accuracy
    predict = []
    accuracy = []
    for yi, score in zip(yb, scores):
        argmax = -1
        valmax = 0
        for idx, scorei in enumerate(score):
            if scorei.data > valmax:
                valmax = scorei.data
                argmax = idx
        predict.append((argmax, yi))
        accuracy.append((argmax == yi))

    acc = sum(accuracy) / len(accuracy)
        
    return total_loss, acc, predict

In [ ]:
# evaluate
loss_all = []
acc_all = []
pred_all = []
for k in range(10):
    total_loss, acc, pred = test(batch_size = BATCH_SIZE)
    loss_all.append(total_loss)
    acc_all.append(acc)
    pred_all.append(pred)
    print(f"eval {k + 1} loss {total_loss.data}, accuracy {acc*100}%")